# Stratify residues into functions based on UniProtKB annotations

In [5]:
# Author: Lisa Boatner
# Date Created: 221128
# Date Modified: 221206
# Updates: 

# Import Modules 

In [6]:
import os, sys
import numpy as np
import pandas as pd
import seaborn as sns
import string
from matplotlib import pyplot as plt

In [12]:
# assuming current directory is main folder
os.chdir('data')
cd = os.getcwd()
cd

'C:\\Users\\Onee-sama\\Documents\\GitHub\\residue_function_annotations\\residue_site_annotations\\data'

In [13]:
# set the date for naming files
date = '2401'

# 

# UniProtKB 

## Download UniProt File with columns: Entry, Active Site, Binding Site, Disulfide Bond, Redox Potential, PDB, Sequence
## https://rest.uniprot.org/uniprotkb/search?fields=accession%2Cid%2Cgene_names%2Cgene_primary%2Cgene_synonym%2Cprotein_name%2Cft_act_site%2Cft_binding%2Cft_dna_bind%2Ccc_catalytic_activity%2Ccc_cofactor%2Cft_disulfid%2Credox_potential%2Cft_site%2Cstructure_3d%2Ccc_function%2Ckeyword%2Csequence&format=xlsx&query=%28Human%29+AND+%28model_organism%3A9606%29+AND+%28reviewed%3Atrue%29&size=500

In [14]:
u_df = pd.read_excel('uniprotkb_Human_AND_model_organism_9606_2024_06_25.xlsx')

C:\ProgramData\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [15]:
u_df.shape

(20435, 18)

In [16]:
u_df.head()

,Entry,Entry Name,Gene Names,Gene Names (primary),Gene Names (synonym),Protein names,Active site,Binding site,DNA binding,Catalytic activity,Cofactor,Disulfide bond,Redox potential,Site,3D,Function [CC],Keywords,Sequence
0,A0A087X1C5,CP2D7_HUMAN,CYP2D7,CYP2D7,NaN,Putative cytochrome P450 2D7 (EC 1.14.14.1),NaN,"BINDING 461; /ligand=""heme""; /ligand_id=""ChEBI...",NaN,CATALYTIC ACTIVITY: Reaction=an organic molecu...,COFACTOR: Name=heme; Xref=ChEBI:CHEBI:30413;,NaN,NaN,NaN,NaN,FUNCTION: May be responsible for the metabolis...,Cytoplasm;Glycoprotein;Heme;Iron;Membrane;Meta...,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...
1,A0A0B4J2F0,PIOS1_HUMAN,PIGBOS1,PIGBOS1,NaN,Protein PIGBOS1 (PIGB opposite strand protein 1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FUNCTION: Plays a role in regulation of the un...,Direct protein sequencing;Membrane;Mitochondri...,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...
2,A0A0B4J2F2,SIK1B_HUMAN,SIK1B,SIK1B,NaN,Putative serine/threonine-protein kinase SIK1B...,"ACT_SITE 149; /note=""Proton acceptor""; /eviden...","BINDING 33..41; /ligand=""ATP""; /ligand_id=""ChE...",NaN,CATALYTIC ACTIVITY: Reaction=ATP + L-seryl-[pr...,COFACTOR: Name=Mg(2+); Xref=ChEBI:CHEBI:18420;...,NaN,NaN,NaN,NaN,FUNCTION: Probable serine/threonine-protein ki...,ATP-binding;Kinase;Magnesium;Metal-binding;Nuc...,MVIMSEFSADPAGQGQGQQKPLRVGFYDIERTLGKGNFAVVKLARH...
3,A0A0C5B5G6,MOTSC_HUMAN,MT-RNR1,MT-RNR1,NaN,Mitochondrial-derived peptide MOTS-c (Mitochon...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FUNCTION: Regulates insulin sensitivity and me...,DNA-binding;Mitochondrion;Nucleus;Osteogenesis...,MRWQEMGYIFYPRKLR
4,A0A0K2S4Q6,CD3CH_HUMAN,CD300H,CD300H,NaN,Protein CD300H (CD300 antigen-like family memb...,NaN,NaN,NaN,NaN,NaN,"DISULFID 43..111; /evidence=""ECO:0000255|PROSI...",NaN,NaN,NaN,FUNCTION: May play an important role in innate...,Alternative splicing;Disulfide bond;Glycoprote...,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...


In [17]:
u_df.columns.to_list()

['Entry',
 'Entry Name',
 'Gene Names',
 'Gene Names (primary)',
 'Gene Names (synonym)',
 'Protein names',
 'Active site',
 'Binding site',
 'DNA binding',
 'Catalytic activity',
 'Cofactor',
 'Disulfide bond',
 'Redox potential',
 'Site',
 '3D',
 'Function [CC]',
 'Keywords',
 'Sequence']

# 

# Get Domain Range

In [43]:
# ACT_SITE 149; /note="Proton acceptor";
# TOPO_DOM 1..24; /note="Extracellular"; /evidence="ECO:0000255"; TOPO_DOM 46..101; /note="Cytoplasmic"; /evidence="ECO:0000255"; TOPO_DOM 123..138; 
def find_domains(df, col, kw, extra):
    
    domains = []
    names = []
    
    regions = df[col].to_list()
    seqs = df['Sequence'].to_list()
    pros = df['Entry'].to_list()
    
    for i in range(len(regions)):
        entry = regions[i].replace('/', '')
        entries = entry.split(';')
        
        skip = False

        for j in range(len(entries)):
            if kw in entries[j]:
                
                nums = entries[j].replace(kw + ' ', '')
                
                if "?" in nums:
                    skip = True
                    continue
                
                elif "-" in nums:
                    skip = True
                    continue
                
                elif ">" in nums:
                    nums = nums.replace('>', '')
                    
                elif ".." in nums:
                    aa_num = nums.split('..')
                    start = aa_num[0]
                    end = aa_num[1]
                else:
                    start = nums.strip()
                    end = nums.strip()
                
                sequence =  seqs[i]

                if (int(start) - 1) > len(sequence):
                    start_aa = '-'
                else:
                    start_aa = sequence[int(start)-1]
                
                if (int(end) - 1) > len(sequence):
                    end_aa = sequence[-1]
                else:
                    end_aa = sequence[int(end)-1]
                
                identifier = pros[i] + '_' + start_aa.strip() + str(start).strip() + '_' + end_aa.strip() + str(end).strip()
                
                domains.append(identifier)
                
            if (extra in entries[j]) & (skip == False):
                note = entries[j].replace(extra + '"', '')[:-1].strip()
                names.append(note)
                
            skip = False
            
        if len(domains) != len(names):
            names.append('')

    return domains, names

In [44]:
def get_domains(df, domain_ty, domain_kw, domain_nm, extra):
    domain_df = df[df[domain_ty].isna() == False]
    
    domains, names = find_domains(domain_df, domain_ty, domain_kw, extra)
    domain_id_df = pd.DataFrame()
    domain_id_df['identifier'] = domains
    domain_id_df[domain_nm] = names
    
    domain_id_df['proteinid'] = domain_id_df['identifier'].map(lambda x: str(x).split('_')[0])
    domain_id_df[domain_nm + '_start'] = domain_id_df['identifier'].map(lambda x: str(x).split('_')[1])
    domain_id_df[domain_nm + '_end'] = domain_id_df['identifier'].map(lambda x: str(x).split('_')[-1])
    
    return domain_id_df

# 

# Active Sites

In [45]:
active_domain_df = get_domains(u_df, 'Active site', 'ACT_SITE', 'active_region', "note=")
active_domain_df.shape

(3582, 5)

In [46]:
active_domain_df.head()

,identifier,active_region,proteinid,active_region_start,active_region_end
0,A0A0B4J2F2_D149_D149,Proton acceptor,A0A0B4J2F2,D149,D149
1,A0A1B0GTW7_E306_E306,,A0A1B0GTW7,E306,E306
2,A0AVT1_C625_C625,Glycyl thioester intermediate,A0AVT1,C625,C625
3,A1KZ92_H812_H812,Proton acceptor,A1KZ92,H812,H812
4,A1L167_C88_C88,Glycyl thioester intermediate,A1L167,C88,C88


In [47]:
active_domain_df.to_csv(date + '_uniprot_active_region_identifiers.csv', index = False)

# 

# Binding Sites 

In [49]:
binding_domain_df = get_domains(u_df, 'Binding site', 'BINDING', 'binding_region', "ligand=")
binding_domain_df.shape

(28192, 5)

In [50]:
binding_domain_df.head()

,identifier,binding_region,proteinid,binding_region_start,binding_region_end
0,A0A087X1C5_C461_C461,heme,A0A087X1C5,C461,C461
1,A0A0B4J2F2_L33_V41,ATP,A0A0B4J2F2,L33,V41
2,A0A0B4J2F2_K56_K56,ATP,A0A0B4J2F2,K56,K56
3,A0A1B0GTW7_H305_H305,Zn(2+),A0A1B0GTW7,H305,H305
4,A0A1B0GTW7_H309_H309,Zn(2+),A0A1B0GTW7,H309,H309


In [51]:
binding_domain_df.to_csv(date + '_uniprot_binding_region_identifiers.csv', index = False)

# 

# Read Residue IDs 

In [52]:
residue_df = pd.read_csv('aa_ids/2401_uniprot_cysteineids.csv')

In [53]:
residue_df.shape

(261951, 2)

In [54]:
residue_df.head()

,proteinid,residueid
0,A0A087X1C5,A0A087X1C5_C57
1,A0A087X1C5,A0A087X1C5_C159
2,A0A087X1C5,A0A087X1C5_C161
3,A0A087X1C5,A0A087X1C5_C191
4,A0A087X1C5,A0A087X1C5_C337


# 

# Which amino acids are in active or binding sites? 

In [61]:
def get_resid(df, col):

    new_df = df.copy()
    vals = []

    for index, row in df.iterrows():
        vals.append(row[col].split('_')[1])

    new_df['resid'] = vals
    return new_df

In [62]:
def in_between(df, col, start, end, site_name):

    new_df = df.copy()
    vals = []
    
    for index, row in df.iterrows():
        current_resid = int(row[col][1:])

        if (current_resid >= int(row[start][1:])) & (current_resid <= int(row[end][1:])):
            vals.append('yes')
            
        else:
            vals.append(None)
            
    new_df['resid_in_' + site_name] = vals
    return new_df

In [79]:
def find_inbetween(input_df, site_df, site_name, site_type):
    subset_input_df = input_df[['proteinid', 'residueid']]
    subset_input_df = subset_input_df.drop_duplicates()
    
    # merge interpro and input
    domain_df = pd.merge(subset_input_df, site_df, on = 'proteinid', how = 'left')
    found_domain_df = domain_df[domain_df['proteinid'].isna() == False]
    print("Merged Sites and Input")

    # create resid
    # found_domain_df['resid'] = found_domain_df[args.rc].map(lambda x: str(x).split('_')[1])
    found_domain_w_resid_df = get_resid(found_domain_df, 'residueid')
    # found_domain_df['resid'] = resid_vals
    print("Created Resid")

    # find residues in interpro domains
    no_na_df = found_domain_w_resid_df[found_domain_w_resid_df[site_name + '_start'].isna() == False]
    no_na_site_df = in_between(no_na_df, 'resid', site_name + '_start', site_name + '_end', site_type)

    # na_df = found_domain_w_resid_df[found_domain_w_resid_df[site_name + '_start'].isna() != False]
    # na_site_df = na_df.copy()
    # na_site_df['resid_in_' + args.sc.strip()] = None

    # site_df = pd.concat([no_na_site_df, na_site_df])
    site_df = no_na_site_df.copy()
    print("Found Resids in UniProt Site Regions")

    # subset output
    subset_found_domain_df = site_df.drop(columns = [site_name + '_start', site_name + '_end'])
    subset_found_domain_df = subset_found_domain_df.drop_duplicates()
    
    print(subset_found_domain_df['resid_in_' + site_type].value_counts())

    return subset_found_domain_df

# 

## Active Sites 

In [73]:
found_active_region_df = find_inbetween(residue_df, active_domain_df, 'active_region', 'ar')

Merged Sites and Input
Created Resid
Found Resids in UniProt Site Regions
yes    595
Name: resid_in_ar, dtype: int64


In [74]:
found_active_region_df.head()

,proteinid,residueid,identifier,active_region,resid,resid_in_ar
9,A0A0B4J2F2,A0A0B4J2F2_C140,A0A0B4J2F2_D149_D149,Proton acceptor,C140,None
10,A0A0B4J2F2,A0A0B4J2F2_C184,A0A0B4J2F2_D149_D149,Proton acceptor,C184,None
11,A0A0B4J2F2,A0A0B4J2F2_C219,A0A0B4J2F2_D149_D149,Proton acceptor,C219,None
12,A0A0B4J2F2,A0A0B4J2F2_C251,A0A0B4J2F2_D149_D149,Proton acceptor,C251,None
13,A0A0B4J2F2,A0A0B4J2F2_C283,A0A0B4J2F2_D149_D149,Proton acceptor,C283,None


In [75]:
found_active_region_df.to_csv(date + '_uniprot_active_region_identifiers.csv', index = False)

# 

# Binding Sites 

In [80]:
found_binding_region_df = find_inbetween(residue_df, binding_domain_df, 'binding_region', 'br')

Merged Sites and Input
Created Resid
Found Resids in UniProt Site Regions
yes    4928
Name: resid_in_br, dtype: int64


In [81]:
found_binding_region_df.head()

,proteinid,residueid,identifier,binding_region,resid,resid_in_br
0,A0A087X1C5,A0A087X1C5_C57,A0A087X1C5_C461_C461,heme,C57,None
1,A0A087X1C5,A0A087X1C5_C159,A0A087X1C5_C461_C461,heme,C159,None
2,A0A087X1C5,A0A087X1C5_C161,A0A087X1C5_C461_C461,heme,C161,None
3,A0A087X1C5,A0A087X1C5_C191,A0A087X1C5_C461_C461,heme,C191,None
4,A0A087X1C5,A0A087X1C5_C337,A0A087X1C5_C461_C461,heme,C337,None


In [82]:
found_binding_region_df.to_csv(date + '_uniprot_binding_region_identifiers.csv', index = False)